In [1]:
conda install pandas

Channels:
 - defaults
 - anaconda
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install matplotlib

Channels:
 - defaults
 - anaconda
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install numpy

Channels:
 - defaults
 - anaconda
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install -c anaconda openpyxl

Channels:
 - anaconda
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [19]:
# Reload the environment and process the file again
import pandas as pd

# Load the Excel file
file_path = "R3 L1 PM cellular distribution_8 bit.xlsx"
excel_data = pd.ExcelFile(file_path)

# Function to add Region_Name and Specific_Region columns
def process_and_fix_regions(file_path):
    summary = []
    for sheet in excel_data.sheet_names:
        sheet_data = pd.read_excel(file_path, sheet_name=sheet)
        
        if 'Max' in sheet_data.columns and 'Channel' in sheet_data.columns and 'Region' in sheet_data.columns:
            # Filter and group data
            filtered_data = sheet_data[sheet_data['Max'] == 255]
            grouped_data = (
                filtered_data.groupby(['Region', 'Channel'])
                .size()
                .reset_index(name='Count')
            )
            grouped_data['Total ROI\'s'] = (
                sheet_data.groupby(['Region', 'Channel'])
                .size()
                .reset_index(name='Total ROI\'s')['Total ROI\'s']
            )
            grouped_data['Percentage'] = (grouped_data['Count'] / grouped_data['Total ROI\'s']) * 100
            grouped_data['Animal'] = sheet
            
            # Process Region_Name and Specific_Region
            grouped_data['Region_Name'] = grouped_data['Region'].str.replace(r'^(LEFT|RIGHT) ', '', regex=True)
            grouped_data['Region_Name'] = grouped_data['Region_Name'].str.replace(r' Combined$', '', regex=True)
            grouped_data['Specific_Region'] = grouped_data['Region'].str.replace(r' Combined$', 'TOTAL', regex=False)
            
            summary.append(grouped_data)
    
    # Combine all sheets into one summary table
    combined_summary = pd.concat(summary, ignore_index=True)
    
    # Add "Total" rows for each specific region (e.g., Core 1, Core 2)
    total_rows = (
        combined_summary
        .groupby(['Animal', 'Channel', 'Region_Name'], as_index=False)
        .agg({
            'Total ROI\'s': 'sum',
            'Count': 'sum'
        })
    )
    # Calculate the percentage for the total rows
    total_rows['Percentage'] = (total_rows['Count'] / total_rows['Total ROI\'s']) * 100
    total_rows['Specific_Region'] = 'Total ' + total_rows['Region_Name']
    
    # Combine back into the summary
    combined_summary = pd.concat([combined_summary, total_rows], ignore_index=True)
    
    # Ensure the correct columns and order
    combined_summary = combined_summary[
        ['Animal', 'Channel', 'Region_Name', 'Specific_Region', 'Total ROI\'s', 'Count', 'Percentage']
    ]
    
    return combined_summary

# Apply the processing function
final_table = process_and_fix_regions(file_path)

# Split the data into separate tables for each channel
d1dr_table_final = final_table[final_table['Channel'] == 'D1DR']
d2dr_table_final = final_table[final_table['Channel'] == 'D2DR']
oxtr_table_final = final_table[final_table['Channel'] == 'OXTR']


# Save the final tables to Excel files for download
output_path_d1dr = "D1DR_ROI_Counts.xlsx"
output_path_d2dr = "D2DR_ROI_Counts.xlsx"
output_path_oxtr = "OXTR_ROI_Counts.xlsx"

d1dr_table_final.to_excel(output_path_d1dr, index=False)
d2dr_table_final.to_excel(output_path_d2dr, index=False)
oxtr_table_final.to_excel(output_path_oxtr, index=False)

output_path_d1dr, output_path_d2dr, output_path_oxtr

from IPython.display import FileLink, display

# Display download links for the saved Excel files
display(FileLink(output_path_d1dr, result_html_prefix="Download D1DR ROI Counts: "))
display(FileLink(output_path_d2dr, result_html_prefix="Download D2DR ROI Counts: "))
display(FileLink(output_path_oxtr, result_html_prefix="Download OXTR ROI Counts: "))


C:\Users\melo1934\D1DR_ROI_Counts.xlsx

C:\Users\melo1934\D2DR_ROI_Counts.xlsx

C:\Users\melo1934\OXTR_ROI_Counts.xlsx